# Customer Churn Analysis
Churn rate, when applied to a customer base, refers to the proportion of contractual customers or subscribers who leave a supplier during a given time period. It is a possible indicator of customer dissatisfaction, cheaper and/or better offers from the competition, more successful sales and/or marketing by the competition, or reasons having to do with the customer life cycle.

Churn is closely related to the concept of average customer life time. For example, an annual churn rate of 25 percent implies an average customer life of four years. An annual churn rate of 33 percent implies an average customer life of three years. The churn rate can be minimized by creating barriers which discourage customers to change suppliers (contractual binding periods, use of proprietary technology, value-added services, unique business models, etc.), or through retention activities such as loyalty programs. It is possible to overstate the churn rate, as when a consumer drops the service but then restarts it within the same year. Thus, a clear distinction needs to be made between "gross churn", the total number of absolute disconnections, and "net churn", the overall loss of subscribers or members. The difference between the two measures is the number of new subscribers or members that have joined during the same period. Suppliers may find that if they offer a loss-leader "introductory special", it can lead to a higher churn rate and subscriber abuse, as some subscribers will sign on, let the service lapse, then sign on again to take continuous advantage of current specials. https://en.wikipedia.org/wiki/Churn_rate

In [1]:
%%capture

# Get our favorite packages from PyPI
! pip install cufflinks

# Import pre-installed packages
import numpy as np
import pandas as pd

# Suppress unwatned warnings
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("requests").setLevel(logging.WARNING)

In [2]:
# Load our favorite visualization library
import os
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Sign into Plotly with masked, encrypted API key

myPlotlyKey = os.environ['SECRET_ENV_BRETTS_PLOTLY_KEY']
py.sign_in(username='bretto777',api_key=myPlotlyKey)


In [3]:
# Load some data
churnDF = pd.read_csv('https://s3-us-west-1.amazonaws.com/dsclouddata/home/jupyter/churn_train.csv', delimiter=',')
churnDF["Churn"] = churnDF["Churn"].replace(to_replace=False, value='Retain')
churnDF["Churn"] = churnDF["Churn"].replace(to_replace=True, value='Churn')
churnDFs = churnDF.sample(frac=0.07) # Sample for speedy viz
churnDF.head(10)

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn
0,ND,84,415,400-7253,no,yes,33,159.1,106,27.05,...,101,12.73,213.4,108,9.60,13.0,18,3.51,1,Retain
1,RI,117,408,370-5042,no,yes,13,207.6,65,35.29,...,77,12.98,232.8,95,10.48,9.7,3,2.62,1,Retain
2,VA,132,510,343-4696,no,no,0,81.1,86,13.79,...,72,20.84,237.0,115,10.67,10.3,2,2.78,0,Retain
3,OK,121,408,364-2495,no,yes,31,237.1,63,40.31,...,117,17.48,196.7,85,8.85,10.1,5,2.73,4,Retain
4,ME,205,510,413-4039,no,yes,24,175.8,139,29.89,...,98,13.18,180.7,64,8.13,7.8,5,2.11,2,Retain
5,MT,116,415,384-5907,no,yes,35,182.8,122,31.08,...,119,18.08,193.8,103,8.72,11.0,2,2.97,1,Retain
6,MN,113,408,417-5146,no,no,0,158.9,137,27.01,...,109,20.64,247.8,97,11.15,6.5,4,1.76,0,Retain
7,MO,74,415,421-2955,no,no,0,172.1,105,29.26,...,99,17.99,182.2,105,8.20,11.6,6,3.13,1,Retain
8,MI,112,510,420-1383,no,no,0,243.4,77,41.38,...,97,15.48,259.2,94,11.66,12.8,2,3.46,1,Retain
9,UT,68,415,403-8916,no,no,0,213.9,112,36.36,...,100,22.14,233.8,97,10.52,8.4,3,2.27,1,Churn


In [5]:
# separate the calls data for plotting


churnDFs = churnDFs[['Account Length','Day Calls','Eve Calls','CustServ Calls','Churn']]

# Create scatter plot matrix of call data
splom = ff.create_scatterplotmatrix(churnDFs, diag='histogram', index='Churn',  
                                  colormap= dict(
                                      Churn = '#9CBEF1',
                                      Retain = '#04367F'
                                      ),
                                  colormap_type='cat',
                                  height=560, width=650,
                                  size=4, marker=dict(symbol='circle'))
py.iplot(splom)

In [6]:

import h2o
from h2o.automl import H2OAutoML
from __future__ import print_function


[WARNING] H2O requires requests module of version 2.10 or newer. You have version 2.4.3.
You can upgrade to the newest version of the module running from the command line
    $ pip2 install --upgrade requests


In [7]:
h2o.connect(ip="35.225.239.147")
#h2o.init(nthreads=2, max_mem_size="7G")

Connecting to H2O server at http://35.225.239.147:54321... successful.


H2O cluster uptime:,4 mins 48 secs
H2O cluster version:,3.16.0.2
H2O cluster version age:,"7 days, 3 hours and 7 minutes"
H2O cluster name:,olmsteadbrett
H2O cluster total nodes:,1
H2O cluster free memory:,10.65 Gb
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://35.225.239.147:54321
H2O connection proxy:,None


<H2OConnection to http://35.225.239.147:54321, session _sid_9de4>

In [8]:
# Split data into training and testing frames

from sklearn import cross_validation
from sklearn.model_selection import train_test_split

training, testing = train_test_split(churnDF, train_size=0.8, stratify=churnDF["Churn"], random_state=9)
train = h2o.H2OFrame(python_obj=training).drop("State")
test = h2o.H2OFrame(python_obj=testing).drop("State")

# Set predictor and response variables
y = "Churn"
x = train.columns
x.remove(y)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


# Automatic Machine Learning
enter some text here

In [9]:
# Run AutoML until 11 models are built
autoModel = H2OAutoML(max_models = 11)
autoModel.train(x = x, y = y,
          training_frame = train,
          validation_frame = test, 
          leaderboard_frame = test)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


## Leaderboard

In [10]:
leaders = autoModel.leaderboard
leaders

model_id,auc,logloss
GBM_grid_0_AutoML_20171207_212250_model_0,0.947606,0.133365
GBM_grid_0_AutoML_20171207_212250_model_2,0.946339,0.138035
StackedEnsemble_AllModels_0_AutoML_20171207_212250,0.944472,0.122702
StackedEnsemble_BestOfFamily_0_AutoML_20171207_212250,0.940438,0.125301
GBM_grid_0_AutoML_20171207_212250_model_3,0.936214,0.141755
GBM_grid_0_AutoML_20171207_212250_model_1,0.935955,0.139211
XRT_0_AutoML_20171207_212250,0.929632,0.255587
GBM_grid_0_AutoML_20171207_212250_model_4,0.926771,0.194585
DRF_0_AutoML_20171207_212250,0.922942,0.521455
DeepLearning_0_AutoML_20171207_212250,0.921729,0.217233


# Variable Importances
Below we plot variable importances as reported by the best performing algo in the ensemble.

In [12]:
importances = h2o.get_model(leaders[1, 0]).varimp(use_pandas=True)
importances = importances.loc[:,['variable','relative_importance']].groupby('variable').mean()
importances.sort_values(by="relative_importance", ascending=False).iplot(kind='bar', colors='#5AC4F2', theme='white')

# Super Model vs the Base models
This plot shows the ROC curves for the Super Model, the Best Base Model, and 9 next best models in the ensemble. 

In [13]:
Model0 = np.array(h2o.get_model(leaders[0,0]).roc(xval=True))
Model1 = np.array(h2o.get_model(leaders[1,0]).roc(xval=True))
Model2 = np.array(h2o.get_model(leaders[2,0]).roc(xval=True))
Model3 = np.array(h2o.get_model(leaders[3,0]).roc(xval=True))
Model4 = np.array(h2o.get_model(leaders[4,0]).roc(xval=True))
Model5 = np.array(h2o.get_model(leaders[5,0]).roc(xval=True))
Model6 = np.array(h2o.get_model(leaders[6,0]).roc(xval=True))
Model7 = np.array(h2o.get_model(leaders[7,0]).roc(xval=True))
Model8 = np.array(h2o.get_model(leaders[8,0]).roc(xval=True))
Model9 = np.array(h2o.get_model(leaders[9,0]).roc(xval=True))

In [14]:
layout = go.Layout(autosize=False, width=725, height=575,  xaxis=dict(title='False Positive Rate', titlefont=dict(family='Arial, sans-serif', size=15, color='grey')), 
                                                           yaxis=dict(title='True Positive Rate', titlefont=dict(family='Arial, sans-serif', size=15, color='grey')))

traceChanceLine = go.Scatter(x = [0,1], y = [0,1], mode = 'lines+markers', name = 'chance', line = dict(color = ('rgb(136, 140, 150)'), width = 4, dash = 'dash'))
Model0Trace = go.Scatter(x = Model0[0], y = Model0[1], mode = 'lines', name = 'Model 0', line = dict(color = ('rgb(26, 58, 126)'), width = 3))
Model1Trace = go.Scatter(x = Model1[0], y = Model1[1], mode = 'lines', name = 'Model 1', line = dict(color = ('rgb(156, 190, 241))'), width = 1))
Model2Trace = go.Scatter(x = Model2[0], y = Model2[1], mode = 'lines', name = 'Model 2', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model3Trace = go.Scatter(x = Model3[0], y = Model3[1], mode = 'lines', name = 'Model 3', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model4Trace = go.Scatter(x = Model4[0], y = Model4[1], mode = 'lines', name = 'Model 4', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model5Trace = go.Scatter(x = Model5[0], y = Model5[1], mode = 'lines', name = 'Model 5', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model6Trace = go.Scatter(x = Model6[0], y = Model6[1], mode = 'lines', name = 'Model 6', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
#Model7Trace = go.Scatter(x = Model7[0], y = Model7[1], mode = 'lines', name = 'Model 7', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model8Trace = go.Scatter(x = Model8[0], y = Model8[1], mode = 'lines', name = 'Model 8', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model9Trace = go.Scatter(x = Model9[0], y = Model9[1], mode = 'lines', name = 'Model 9', line = dict(color = ('rgb(156, 190, 241)'), width = 1))

fig = go.Figure(data=[Model0Trace,Model1Trace,Model2Trace,Model3Trace,Model4Trace,Model5Trace,Model6Trace,Model8Trace,Model9Trace,traceChanceLine], layout=layout)

py.iplot(fig)

# Confusion Matrix

In [16]:
cm = h2o.get_model(leaders[1, 0]).confusion_matrix(xval=True)
cm = cm.table.as_data_frame()
cm
confusionMatrix = ff.create_table(cm)
confusionMatrix.layout.height=300
confusionMatrix.layout.width=800
confusionMatrix.layout.font.size=17
py.iplot(confusionMatrix)

# Business Impact Matrix

Weighting Predictions With a Dollar Value
-   Correctly predicting retain: `+$5`
-   Correctly predicting churn: `+$75`
-   Incorrectly predicting retain: `-$150`
-   Incorrectly predicting churn: `-$1.5`

    

In [17]:
CorrectPredictChurn = cm.loc[0,'Churn']
CorrectPredictChurnImpact = 75
cm1 = CorrectPredictChurn*CorrectPredictChurnImpact

IncorrectPredictChurn = cm.loc[1,'Churn']
IncorrectPredictChurnImpact = -5
cm2 = IncorrectPredictChurn*IncorrectPredictChurnImpact

IncorrectPredictRetain = cm.loc[0,'Retain']
IncorrectPredictRetainImpact = -150
cm3 = IncorrectPredictRetain*IncorrectPredictRetainImpact

CorrectPredictRetain = cm.loc[0,'Retain']
CorrectPredictRetainImpact = 5
cm4 = IncorrectPredictRetain*CorrectPredictRetainImpact


data_matrix = [['Business Impact', '($) Predicted Churn', '($) Predicted Retain', '($) Total'],
               ['($) Actual Churn', cm1, cm3, '' ],
               ['($) Actual Retain', cm2, cm4, ''],
               ['($) Total', cm1+cm2, cm3+cm4, cm1+cm2+cm3+cm4]]

impactMatrix = ff.create_table(data_matrix, height_constant=20, hoverinfo='weight')
impactMatrix.layout.height=300
impactMatrix.layout.width=800
impactMatrix.layout.font.size=17
py.iplot(impactMatrix)

In [24]:
cm

,,Churn,Retain,Error,Rate
0,Churn,225.0,96.0,0.2991,(96.0/321.0)
1,Retain,13.0,1798.0,0.0072,(13.0/1811.0)
2,Total,238.0,1894.0,0.0511,(109.0/2132.0)


In [18]:
print("Total customers evaluated: 2132")

Total customers evaluated: 2132


In [19]:
print("Total value created by the model: $" + str(cm1+cm2+cm3+cm4))

Total value created by the model: $2890.0


In [25]:
print("Total value per customer: $" +str(round(((cm1+cm2+cm3+cm4)/2132),3)))

Total value per customer: $1.356


In [26]:
os.getcwd()

'/home/datascience/Predict-Churn'

In [31]:
h2o.get_model(leaders[1, 0])
h2o.save_model()

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_0_AutoML_20171207_212250_model_2


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.00699602637897
RMSE: 0.0836422523547
LogLoss: 0.0413619376734
Mean Per-Class Error: 0.00366744591291
AUC: 0.999926031813
Gini: 0.999852063626
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.726316701466: 


,Churn,Retain,Error,Rate
Churn,317.0,4.0,0.0125,(4.0/321.0)
Retain,0.0,1811.0,0.0,(0.0/1811.0)
Total,317.0,1815.0,0.0019,(4.0/2132.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.7263167,0.9988969,224.0
max f2,0.7263167,0.9995585,224.0
max f0point5,0.8131808,0.9988956,220.0
max accuracy,0.8131808,0.9981238,220.0
max precision,0.9981563,1.0,0.0
max recall,0.7263167,1.0,224.0
max specificity,0.9981563,1.0,0.0
max absolute_mcc,0.8131808,0.9926651,220.0
max min_per_class_accuracy,0.8520386,0.9955826,213.0
max mean_per_class_accuracy,0.8131808,0.9963326,220.0


Gains/Lift Table: Avg response rate: 84.94 %



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.9512243,0.0061256,0.9578454,0.9344263,0.9577465,0.9530516,0.9530516
auc,0.9106764,0.0072558,0.909966,0.8951581,0.9202335,0.9232736,0.9047508
err,0.0487757,0.0061256,0.0421546,0.0655738,0.0422535,0.0469484,0.0469484
err_count,20.8,2.6229753,18.0,28.0,18.0,20.0,20.0
f0point5,0.9609789,0.0071671,0.9690553,0.9422975,0.9707950,0.9601936,0.9625527
f1,0.9718401,0.0035097,0.9754099,0.9621621,0.9755435,0.9727520,0.9733334
f2,0.9830141,0.0013135,0.9818482,0.9828824,0.9803386,0.9856433,0.9843581
lift_top_group,1.1773996,0.0096749,1.179558,1.1960784,1.1671233,1.1866295,1.1576087
logloss,0.1823291,0.0164561,0.1663252,0.2275895,0.1674323,0.1814096,0.1688888
max_per_class_error,0.2688392,0.0491817,0.2,0.3857143,0.1967213,0.2686567,0.2931035


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2017-12-07 21:23:07,2.425 sec,0.0,0.3576223,0.4236842,0.5,1.0,0.1505629,0.3587174,0.4256264,0.5,1.0,0.1516854
,2017-12-07 21:23:07,2.452 sec,5.0,0.2751751,0.2705003,0.9617877,1.1772501,0.0389306,0.2854099,0.2887799,0.9098466,1.1788079,0.0580524
,2017-12-07 21:23:07,2.482 sec,10.0,0.2235026,0.2014110,0.9757470,1.1772501,0.0267355,0.2428912,0.2273469,0.9168234,1.1788079,0.0393258
,2017-12-07 21:23:07,2.516 sec,15.0,0.1917392,0.1608863,0.9825917,1.1772501,0.0248593,0.2182852,0.1937716,0.9202300,1.1788079,0.0393258
,2017-12-07 21:23:07,2.549 sec,20.0,0.1717266,0.1346231,0.9859366,1.1772501,0.0225141,0.2037593,0.1739608,0.9281198,1.1788079,0.0355805
,2017-12-07 21:23:07,2.584 sec,25.0,0.1566266,0.1148757,0.9896427,1.1772501,0.0215760,0.1926221,0.1602979,0.9263483,1.1788079,0.0337079
,2017-12-07 21:23:07,2.613 sec,30.0,0.1454960,0.1006014,0.9911599,1.1772501,0.0187617,0.1859186,0.1503661,0.9331480,1.1788079,0.0337079
,2017-12-07 21:23:07,2.645 sec,35.0,0.1340498,0.0857440,0.9950381,1.1772501,0.0159475,0.1850093,0.1476129,0.9310904,1.1788079,0.0337079
,2017-12-07 21:23:07,2.677 sec,40.0,0.1203416,0.0716827,0.9979220,1.1772501,0.0084428,0.1816864,0.1420406,0.9383942,1.1788079,0.0318352
,2017-12-07 21:23:07,2.709 sec,45.0,0.1098361,0.0619680,0.9987855,1.1772501,0.0051595,0.1824053,0.1402476,0.9438858,1.1788079,0.0337079


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
Day Mins,191.6935577,1.0,0.1807785
CustServ Calls,132.3271484,0.6903057,0.1247925
Day Charge,125.1529007,0.6528801,0.1180267
Eve Charge,99.7912827,0.5205771,0.0941092
Intl Calls,74.7870026,0.3901383,0.0705286
Int'l Plan,64.8073044,0.3380776,0.0611172
Eve Mins,59.9713516,0.3128501,0.0565566
Intl Charge,59.3775482,0.3097524,0.0559966
Night Charge,36.5123215,0.1904723,0.0344333
VMail Message,35.3229752,0.1842679,0.0333117


In [34]:
# Save the best model
h2o.download_pojo(model=h2o.get_model(leaders[1, 0]))
#path = h2o.save_model(model=h2o.get_model(leaders[1, 0]), force=True)
#os.rename(autoMLleader.model_id, "AutoML-leader")    

/*
  Licensed under the Apache License, Version 2.0
    http://www.apache.org/licenses/LICENSE-2.0.html

  AUTOGENERATED BY H2O at 2017-12-07T21:56:42.756Z
  3.16.0.2
  
  Standalone prediction code with sample test data for GBMModel named GBM_grid_0_AutoML_20171207_212250_model_2

  How to download, compile and execute:
      mkdir tmpdir
      cd tmpdir
      curl http://10.128.0.4:54321/3/h2o-genmodel.jar > h2o-genmodel.jar
      curl http://10.128.0.4:54321/3/Models.java/GBM_grid_0_AutoML_20171207_212250_model_2 > GBM_grid_0_AutoML_20171207_212250_model_2.java
      javac -cp h2o-genmodel.jar -J-Xmx2g -J-XX:MaxPermSize=128m GBM_grid_0_AutoML_20171207_212250_model_2.java

     (Note:  Try java argument -XX:+PrintCompilation to show runtime JIT compiler behavior.)
*/
import java.util.Map;
import hex.genmodel.GenModel;
import hex.genmodel.annotations.ModelPojo;

@ModelPojo(name="GBM_grid_0_AutoML_20171207_212250_model_2", algorithm="gbm")
public class GBM_grid_0_AutoML_20171207_212250

In [50]:
LoadedEnsemble = h2o.load_model(path="AutoML-leader")
print(LoadedEnsemble)

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid__820851e96ed01c37adaba6babe314554_model_4


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0234803483569
RMSE: 0.15323298717
LogLoss: 0.121329793338
Mean Per-Class Error: 0.0
AUC: 1.0
Gini: 1.0
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.88118493422: 


,Churn,Retain,Error,Rate
Churn,321.0,0.0,0.0,(0.0/321.0)
Retain,0.0,1811.0,0.0,(0.0/1811.0)
Total,321.0,1811.0,0.0,(0.0/2132.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.8811849,1.0,225.0
max f2,0.8811849,1.0,225.0
max f0point5,0.8811849,1.0,225.0
max accuracy,0.8811849,1.0,225.0
max precision,0.9474707,1.0,0.0
max recall,0.8811849,1.0,225.0
max specificity,0.9474707,1.0,0.0
max absolute_mcc,0.8811849,1.0,225.0
max min_per_class_accuracy,0.8811849,1.0,225.0
max mean_per_class_accuracy,0.8811849,1.0,225.0


Gains/Lift Table: Avg response rate: 84.94 %



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.9498169,0.0047315,0.9531616,0.9367682,0.9553990,0.9530516,0.9507042
auc,0.9019006,0.0123003,0.9198257,0.8803121,0.9248372,0.8896396,0.8948885
err,0.0501831,0.0047315,0.0468384,0.0632319,0.0446009,0.0469484,0.0492958
err_count,21.4,2.0297782,20.0,27.0,19.0,20.0,21.0
f0point5,0.9591722,0.0057986,0.9663410,0.9442971,0.9643048,0.9645777,0.9563410
f1,0.9710712,0.0027431,0.9726776,0.9634641,0.974428,0.9725274,0.9722589
f2,0.9833229,0.0023746,0.9790979,0.9834254,0.9847661,0.9806094,0.9887158
lift_top_group,1.1773996,0.0096749,1.179558,1.1960784,1.1671233,1.1866295,1.1576087
logloss,0.2236371,0.0167864,0.2083235,0.2702373,0.2057928,0.2181397,0.215692
max_per_class_error,0.2870116,0.0474201,0.2153846,0.3714286,0.2622951,0.2238806,0.3620690


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2017-11-08 14:27:05,36.011 sec,0.0,0.3576223,0.4236842,0.5,1.0,0.1505629,0.3587174,0.4256264,0.5,1.0,0.1516854
,2017-11-08 14:27:05,36.060 sec,5.0,0.3422073,0.3847247,0.9968245,1.1772501,0.0103189,0.3459903,0.3932781,0.9361459,1.1788079,0.0393258
,2017-11-08 14:27:05,36.135 sec,10.0,0.3273413,0.3528530,0.9989137,1.1772501,0.0037523,0.3343219,0.3679579,0.9466792,1.1788079,0.0355805
,2017-11-08 14:27:06,36.222 sec,15.0,0.3135773,0.3268003,0.9993326,1.1772501,0.0032833,0.3240986,0.3482015,0.9405609,1.1788079,0.0355805
,2017-11-08 14:27:06,36.331 sec,20.0,0.3001654,0.3037479,0.9995201,1.1772501,0.0023452,0.3145189,0.3312580,0.9373177,1.1788079,0.0374532
---,---,---,---,---,---,---,---,---,---,---,---,---,---
,2017-11-08 14:27:08,38.704 sec,85.0,0.1781354,0.1472527,1.0,1.1772501,0.0,0.2355400,0.2162149,0.9454937,1.1788079,0.0393258
,2017-11-08 14:27:08,38.964 sec,90.0,0.1713121,0.1400315,1.0,1.1772501,0.0,0.2320689,0.2113156,0.9448124,1.1788079,0.0393258
,2017-11-08 14:27:09,39.236 sec,95.0,0.1651937,0.1336234,1.0,1.1772501,0.0,0.2286997,0.2065011,0.9474014,1.1788079,0.0393258
,2017-11-08 14:27:09,39.523 sec,100.0,0.1590832,0.1273052,1.0,1.1772501,0.0,0.2257222,0.2022857,0.9466383,1.1788079,0.0393258



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
Day Mins,1452.4663086,1.0,0.1460925
Day Charge,1306.8050537,0.8997145,0.1314415
CustServ Calls,1181.8673096,0.8136969,0.1188750
Intl Calls,816.2738647,0.5619916,0.0821027
Eve Charge,725.2891235,0.4993500,0.0729513
Eve Mins,674.5635986,0.4644263,0.0678492
Int'l Plan,674.2656860,0.4642212,0.0678192
Intl Charge,527.4998169,0.3631753,0.0530572
Intl Mins,383.8133545,0.2642494,0.0386049
VMail Message,342.9296875,0.2361016,0.0344927
